In [ ]:
// Kotlin notebook
import jline.*
import jline.lang.*
import jline.lang.nodes.*
import jline.lang.processes.*
import jline.lang.constant.*
import jline.solvers.ctmc.*
import jline.solvers.jmt.*
import jline.solvers.ssa.*
import jline.solvers.fluid.*
import jline.solvers.mva.*
import jline.solvers.nc.*
import jline.solvers.mam.*
import jline.solvers.*
import jline.util.matrix.*

In [ ]:
val model = Network("model")

// Create nodes
val delay = Delay(model, "Delay")
val queue1 = Queue(model, "Queue1", SchedStrategy.PS)
queue1.setNumberOfServers(2)  // Multi-server queue

// Create three closed job classes with different populations
val jobclass1 = ClosedClass(model, "Class1", 2, delay, 0)  // 2 jobs
val jobclass2 = ClosedClass(model, "Class2", 0, delay, 0)  // 0 jobs (created via class switching)
val jobclass3 = ClosedClass(model, "Class3", 1, delay, 0)  // 1 job

In [ ]:
// Set service processes with different distributions
// Delay station service times
delay.setService(jobclass1, Erlang(3, 2))                    // Erlang(3,2)
delay.setService(jobclass2, HyperExp(0.5, 3.0, 10.0))       // HyperExp(0.5,3,10)
delay.setService(jobclass3, Exp(1.0))                       // Exp(1)

// Queue1 station service times
queue1.setService(jobclass1, HyperExp(0.1, 1.0, 10.0))      // HyperExp(0.1,1,10)
queue1.setService(jobclass2, Exp(2.0))                      // Exp(2)
queue1.setService(jobclass3, Exp(3.0))                      // Exp(3)

In [ ]:
// Set complex routing matrix with class switching
val P = model.initRoutingMatrix()
val M = model.getNumberOfStations()

// Class1 routing patterns
// P{1,1} = [0.3,0.1; 0.2,0]
P.set(jobclass1, jobclass1, delay, delay, 0.3)   // Stay at Delay (30%)
P.set(jobclass1, jobclass1, delay, queue1, 0.1)  // Delay -> Queue1 (10%)
P.set(jobclass1, jobclass1, queue1, delay, 0.2)  // Queue1 -> Delay (20%)
P.set(jobclass1, jobclass1, queue1, queue1, 0.0)

// Class1 -> Class2 switching
// P{1,2} = [0.6,0; 0.8,0]
P.set(jobclass1, jobclass2, delay, delay, 0.6)   // Class1 -> Class2 at Delay (60%)
P.set(jobclass1, jobclass2, delay, queue1, 0.0)
P.set(jobclass1, jobclass2, queue1, delay, 0.8)  // Class1 -> Class2 at Queue1 -> Delay (80%)
P.set(jobclass1, jobclass2, queue1, queue1, 0.0)

// Class2 routing patterns  
// P{2,1} = [0,0; 1,0]
P.set(jobclass2, jobclass1, delay, delay, 0.0)
P.set(jobclass2, jobclass1, delay, queue1, 0.0)
P.set(jobclass2, jobclass1, queue1, delay, 1.0)  // Class2 -> Class1 at Queue1 -> Delay (100%)
P.set(jobclass2, jobclass1, queue1, queue1, 0.0)

// P{2,2} = [0,1; 0,0]
P.set(jobclass2, jobclass2, delay, delay, 0.0)
P.set(jobclass2, jobclass2, delay, queue1, 1.0)  // Delay -> Queue1 (100%)
P.set(jobclass2, jobclass2, queue1, delay, 0.0)
P.set(jobclass2, jobclass2, queue1, queue1, 0.0)

// Class3 routing (simple cyclic) - circul(M)
val circulMatrix = Matrix.circul(M)
P.set(jobclass3, jobclass3, delay, delay, circulMatrix.get(0, 0))
P.set(jobclass3, jobclass3, delay, queue1, circulMatrix.get(0, 1))
P.set(jobclass3, jobclass3, queue1, delay, circulMatrix.get(1, 0))
P.set(jobclass3, jobclass3, queue1, queue1, circulMatrix.get(1, 1))

model.link(P)

In [ ]:
// Create array of different solvers to compareval solvers = arrayOf(    CTMC(model),    JMT(model, "seed", 23000, "samples", 5000, "verbose", true),    SSA(model, "seed", 23000, "samples", 5000, "verbose", true),    FLD(model),    MVA(model),    NC(model, "method", "exact"),    MAM(model),    LINE(model, "seed", 23000))// Execute each solver and display resultsfor (solver in solvers) {        try {        val avgTable = solver.getAvgTable()        avgTable.print()                // Also get chain and system tables if available        try {            val avgChainTable = solver.getAvgChainTable()            val avgSysTable = solver.getAvgSysTable()            println("Chain table:")            avgChainTable.print()            println("System table:")            avgSysTable.print()        } catch (e: Exception) {            // Chain/System tables not available for all solvers        }    } catch (e: Exception) {        println("Error with ${solver.getName()}: ${e.message}")    }}

This complex three-class model demonstrates:

1. **Multiple Job Classes**:
   - Class1: 2 jobs, Erlang service at delay
   - Class2: 0 initial jobs (created via switching), HyperExp service
   - Class3: 1 job, simple exponential service

2. **Class Switching**:
   - Class1 can switch to Class2 (60% at delay, 80% at queue)
   - Class2 can switch back to Class1 (100% when leaving queue)
   - Class3 maintains its identity (no switching)

3. **Complex Service Distributions**:
   - **Erlang**: Lower variability service
   - **Hyperexponential**: High variability service
   - **Exponential**: Standard memoryless service

4. **Multi-server Queue**: 2 servers with PS scheduling

5. **Routing Complexity**:
   - Probabilistic routing within classes
   - Class switching probabilities
   - Different routing patterns per class

The model represents a system where:
- Jobs can change their characteristics (class switching)
- Different job types have different service requirements
- Resource sharing occurs at the multi-server queue
- Various analytical and simulation methods validate results